In [10]:
import bempp.api
import numpy as np
import scipy
from numba import objmode
import numba
import cmath
import math
%matplotlib inline
import matplotlib.pyplot as plt
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
bempp.core.opencl_kernels.set_default_device(0,0)

In [11]:
def normalized_spherical_harmonics(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)

def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*wavenumber*r))*scipy.special.jv(q+0.5, wavenumber*r)


def regular_spherical_wavefunctions(p,q,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,x)*normalized_spherical_harmonics(p,q,x)

In [12]:
def coeff_sph_expansion(p,q):
    """Analytical formula of the coefficients of the spherical expansions of plane waves"""
    coeff = 4*np.pi*((1j)**q)*normalized_spherical_harmonics(-p,q,d)/(-1)**(abs(p))
    return coeff

In [13]:
d = [1,0,0]
deg = 10
h = 0.1
#values = []

In [ ]:
# scatterer: sphere
two_branches = bempp.api.shapes.sphere(r = 0.2, origin=(0,0,0), h = h)
space_two_branches = bempp.api.function_space(two_branches,'P',1)

unit_sphere = bempp.api.shapes.sphere(r = 1, origin=(0,0,0), h = h)
vert_unit_sphere = unit_sphere.vertices
space_unit_sphere = bempp.api.function_space(unit_sphere, 'P', 1)

phase_list = []
term1_list = []

for wavenumber in range(10,21):

    eta = wavenumber

    identity2 = bempp.api.operators.boundary.sparse.identity(space_two_branches, space_two_branches,space_two_branches)
    slp2 = bempp.api.operators.boundary.helmholtz.single_layer(space_two_branches,space_two_branches,space_two_branches,wavenumber)
    dlp2 = bempp.api.operators.boundary.helmholtz.double_layer(space_two_branches,space_two_branches,space_two_branches,wavenumber)
    slp_far2 = helmholtz_farfield.single_layer(space_two_branches, vert_unit_sphere, wavenumber)
    dlp_far2 = helmholtz_farfield.double_layer(space_two_branches, vert_unit_sphere,wavenumber)

    lhs2 = 0.5 * identity2 + dlp2 - 1j* eta *slp2

    sph_harm_coeff = []
    slp_far_field_coeff = []
    for q in range(deg+1):
        for p in range(-q,q+1):
            @bempp.api.complex_callable
            def sph_harm(x,n,domain_index,result):
                with objmode():
                    result[0] = normalized_spherical_harmonics(p,q,x)
            sph_harm_grid = bempp.api.GridFunction(space_unit_sphere,fun = sph_harm)
            sph_harm_coeff.append(np.conj(sph_harm_grid.coefficients)/((-1j)**(q+1)))

            @bempp.api.complex_callable
            def dirichlet_fun(x,n,domain_index,result):
                    with objmode():
                        result[0] = - regular_spherical_wavefunctions(p,q,x)

            rhs_fun = bempp.api.GridFunction(space_two_branches, fun = dirichlet_fun)
            field, info = bempp.api.linalg.gmres(lhs2, rhs_fun, tol=1E-5)
            slp_far_field_coeff.append(dlp_far2.evaluate(field) - 1j * eta * slp_far2.evaluate(field))

    mass_mat = bempp.api.operators.boundary.sparse.identity(space_unit_sphere,space_unit_sphere,space_unit_sphere).weak_form().A

    T_matrix = np.zeros(((deg+1)**2,(deg+1)**2),dtype = complex)
    for i in range((deg+1)**2):
        for j in range((deg+1)**2):
            T_matrix[i,j] = (slp_far_field_coeff[j] @ mass_mat @ sph_harm_coeff[i])*wavenumber

    coeff_inc = []
    for q in range(deg+1):
        for p in range(-q, q+1):
            coeff_inc.append(coeff_sph_expansion(p,q))

    coeff_sca = T_matrix @ coeff_inc

    S_matrix = np.identity(T_matrix.shape[1]) + 2* T_matrix
    phase_list.append(cmath.phase(np.linalg.det(S_matrix)))
    term1_list.append( 1/(3*np.pi) * ((wavenumber)**3) * ((4/3) * np.pi*0.2**3) )
    print(wavenumber, cmath.phase(np.linalg.det(S_matrix)),  1/(3*np.pi) * ((wavenumber)**3) * ((4/3) * np.pi*0.2**3) )

In [9]:
for i in range(1,21):
    wavenumber = i
    print ((1/8)* (4 * np.pi * 0.2**2) * (wavenumber**2))

0.06283185307179587
0.25132741228718347
0.5654866776461628
1.0053096491487339
1.5707963267948968
2.261946710584651
3.0787608005179976
4.0212385965949355
5.089380098815465
6.283185307179587
7.6026542216873
9.047786842338605
10.618583169133501
12.31504320207199
14.13716694115407
16.084954386379742
18.158405537749005
20.35752039526186
22.68229895891831
25.13274122871835


In [ ]:
term2_list = []
sum_list = [a + b for a, b in zip(term1_list, term2_list)]
print(sum_list)